In [1]:
#Standard Imports
import os
import sys

In [2]:
#Custom Imports
sys.path.append('../')
from src.chord import Chord
from src.musical_work_input import MusicalWorkInput
from src.music_functions import *
from src.evaluate_bach import *
from src.midi_processing import *

In [3]:
# Importing Chord Vocabulary
chord_df_major = pd.read_csv("../data/chord_vocabulary_major.csv", index_col = 0)
chord_df_minor = pd.read_csv("../data/chord_vocabulary_minor.csv", index_col = 0)
chord_vocab_major, chord_vocab_minor = [], []
for index, name, note_intervals in chord_df_major.itertuples():
    chord_vocab_major.append(Chord(index, name, [int(x) for x in note_intervals.split(',')]))
for index, name, note_intervals in chord_df_minor.itertuples():
    chord_vocab_minor.append(Chord(index, name, [int(x) for x in note_intervals.split(',')]))

In [4]:
# Defining penalties for chord progression
penalties_chord_progression_major = pd.read_csv("../data/chord_progression_major.csv", header = 1, index_col = 0)
penalties_chord_progression_minor = pd.read_csv("../data/chord_progression_minor.csv", header = 1, index_col = 0)
penalties_chord_progression_major = dict(penalties_chord_progression_major.stack())
penalties_chord_progression_minor = dict(penalties_chord_progression_minor.stack())

In [5]:
# Importing Weights
weight_df = pd.read_csv("../data/soft_constraint_weights_temp.csv")

# Defining dictionary of weights for each soft constraint option:
soft_constraint_w_weights = {}
for _, name, w in weight_df.itertuples(): #name population is same as soft_constraint_options
    soft_constraint_w_weights[name] = float(w)
assert sum(v for v in soft_constraint_w_weights.values() if v > 0) == 100

In [6]:
# Defining dictionary of hard and soft constraint options:
hard_constraint_options = ['musical input', 'voice range', 'chord membership', 'first last chords',
                           'chord repetition', 'chord bass repetition', 'adjacent bar chords', 'voice crossing', 'parallel movement',
                          'chord spacing', 'incomplete chord', 'chord distribution']
soft_constraint_options = ['chord progression', 'chord repetition', 'chord bass repetition', 'leap resolution',
                           'melodic movement', 'note repetition', 'parallel movement', 'voice overlap', 'adjacent bar chords',
                           'chord spacing', 'distinct notes', 'incomplete chord', 'voice crossing', 'voice range',
                           'second inversion', 'first inversion', 'chord distribution']

In [7]:
# Defining which hard constraints to use
hard_constraints = {x: True if x in ['musical input', 'voice range', 'chord membership', 'first last chords',
                                     'voice crossing', 'parallel movement',
                                     'chord spacing', 'incomplete chord'] else False for x in hard_constraint_options}

In [8]:
# Test Case 1
key = 2
tonality = 'minor'
meter = 4
first_on_beat = 2
mode = 'D'

In [9]:
if tonality == 'major':
    penalties_chord_progression = penalties_chord_progression_major
    chord_vocab = chord_vocab_major
else:
    penalties_chord_progression = penalties_chord_progression_minor
    chord_vocab = chord_vocab_minor 

In [10]:
# Importing Test Case 1
test_case = pd.read_csv("../data/Ach_Gott_erhor'_mein_Seufzen_und_Wehklagen_transcription.csv", skiprows = 5, header = None, index_col = 0)
list_x = list(test_case.iloc[:-1,-1].apply(lambda y: y.split(', ')))
list_x = [[int(x) for x in i] for i in list_x]
list_c_names = test_case.loc['Chords'].apply(lambda y: y.replace('\'','').split(', ')).iloc[0]
list_c = []
for chord_name in list_c_names:
    exists = False
    for chord in chord_vocab:
        if chord.name == chord_name:
            exists = True
            list_c.append(chord.index)
            break
    if not exists:
        list_c.append(-1)

In [11]:
bach_cost = evaluate_cost(list_x, list_c, key, tonality, meter, first_on_beat, mode,
                  chord_vocab,
                  penalties_chord_progression,
                  hard_constraints, 1000,
                  soft_constraint_w_weights)

In [12]:
#[x[43] for x in list_x]

In [13]:
#[(i,c) for i, c in enumerate(bach_cost['hard constraint incomplete chord']) if c > 0]

In [14]:
for k, v in bach_cost.items():
    try:
        print(k, sum(v))
    except:
        print(k, v)

hard constraint voice range 0
hard constraint chord membership 6000
hard constraint first last chords 0
hard constraint chord repetition 0
hard constraint chord bass repetition 0
hard constraint adjacent bar chords 0
hard constraint voice crossing 1000
hard constraint parallel movement 1000
hard constraint chord spacing 1000
hard constraint incomplete chord 1000
hard constraint chord distribution 0
soft constraint chord progression 456.5
soft constraint chord repetition 20.0
soft constraint chord bass repetition 6.0
soft constraint leap resolution 0
soft constraint melodic movement 139.58333333333334
soft constraint note repetition 9.0
soft constraint parallel movement 0
soft constraint voice overlap 3.0
soft constraint adjacent bar chords 0
soft constraint chord spacing 0
soft constraint distinct notes 72.0
soft constraint incomplete chord 0
soft constraint voice crossing 0
soft constraint voice range 4.0
soft constraint second inversion 26.0
soft constraint first inversion 13.0
soft 

In [15]:
total_bach_cost = sum(sum(v) if 'hard constraint' not in k and isinstance(v, list) else v if 'hard constraint' not in k else 0 for k, v in bach_cost.items())
print(total_bach_cost)
bach_cost_4 = sum(sum(v[-4:]) if 'hard constraint' not in k and isinstance(v, list) else v if 'hard constraint' not in k else 0 for k, v in bach_cost.items())
print(bach_cost_4)
bach_cost_8 = sum(sum(v[-8:]) if 'hard constraint' not in k and isinstance(v, list) else v if 'hard constraint' not in k else 0 for k, v in bach_cost.items())
print(bach_cost_8)
bach_cost_16 = sum(sum(v[-16:]) if 'hard constraint' not in k and isinstance(v, list) else v if 'hard constraint' not in k else 0 for k, v in bach_cost.items())
print(bach_cost_16)
print('Bach violated 2 hard constraints: chord spacing (j = 7) and voice crossing (j = 13)')

812.0833333333334
36.0
112.41666666666667
309.5
Bach violated 2 hard constraints: chord spacing (j = 7) and voice crossing (j = 13)


In [16]:
# Test Case 2
key = 0
tonality = 'major'
meter = 4
first_on_beat = 2
mode = 'D'

In [17]:
if tonality == 'major':
    penalties_chord_progression = penalties_chord_progression_major
    chord_vocab = chord_vocab_major
else:
    penalties_chord_progression = penalties_chord_progression_minor
    chord_vocab = chord_vocab_minor 

In [18]:
# Importing Test Case 2
test_case = pd.read_csv("../data/Ach_Gott_und_Herr_wie_gross_und_schwer_(part)_transcription.csv", skiprows = 5, header = None, index_col = 0)
list_x = list(test_case.iloc[:-1,-1].apply(lambda y: y.split(', ')))
list_x = [[int(x) for x in i] for i in list_x]
list_c_names = test_case.loc['Chords'].apply(lambda y: y.replace('\'','').split(', ')).iloc[0]
list_c = []
for chord_name in list_c_names:
    exists = False
    for chord in chord_vocab:
        if chord.name == chord_name:
            exists = True
            list_c.append(chord.index)
            break
    if not exists:
        list_c.append(-1)

In [19]:
bach_cost = evaluate_cost(list_x, list_c, key, tonality, meter, first_on_beat, mode,
                  chord_vocab,
                  penalties_chord_progression,
                  hard_constraints, 1000,
                  soft_constraint_w_weights)

In [20]:
for k, v in bach_cost.items():
    try:
        print(k, sum(v))
    except:
        print(k, v)

hard constraint voice range 0
hard constraint chord membership 4000
hard constraint first last chords 1000
hard constraint chord repetition 0
hard constraint chord bass repetition 0
hard constraint adjacent bar chords 0
hard constraint voice crossing 2000
hard constraint parallel movement 0
hard constraint chord spacing 1000
hard constraint incomplete chord 2000
hard constraint chord distribution 0
soft constraint chord progression 143.0
soft constraint chord repetition 8.0
soft constraint chord bass repetition 3.0
soft constraint leap resolution 0
soft constraint melodic movement 43.33333333333333
soft constraint note repetition 6.0
soft constraint parallel movement 0
soft constraint voice overlap 6.0
soft constraint adjacent bar chords 0
soft constraint chord spacing 0
soft constraint distinct notes 18.0
soft constraint incomplete chord 0
soft constraint voice crossing 0
soft constraint voice range 0.0
soft constraint second inversion 26.0
soft constraint first inversion 5.0
soft con

In [21]:
#[(i,c) for i, c in enumerate(bach_cost['hard constraint incomplete chord']) if c > 0]

In [22]:
bach_cost_4 = sum(sum(v[-4:]) if 'hard constraint' not in k and isinstance(v, list) else v if 'hard constraint' not in k else 0 for k, v in bach_cost.items())
print(bach_cost_4)
bach_cost_8 = sum(sum(v[-8:]) if 'hard constraint' not in k and isinstance(v, list) else v if 'hard constraint' not in k else 0 for k, v in bach_cost.items())
print(bach_cost_8)
bach_cost_16 = sum(sum(v) if 'hard constraint' not in k and isinstance(v, list) else v if 'hard constraint' not in k else 0 for k, v in bach_cost.items())
print(total_bach_cost)

print('Bach violated 3 hard constraints: voice crossing (j = 11, j = 12) and chord spacing (j = 12)')

42.25
163.83333333333334
812.0833333333334
Bach violated 3 hard constraints: voice crossing (j = 11, j = 12) and chord spacing (j = 12)


In [23]:
#bach_cost

In [24]:
#bach_cost_chord_membership

In [25]:
#array_to_midi(list_x, [20]*4, beat = 600, dest_file_path = '../outputs/bach_Ach bleib\' bei unsm Herr Jesu Christ (part).mid', held_notes = False, offset = 3)